# Evaluation on public dataset: StanWiFi

This notebook uses the StanWiFi dataset to evaluate a preprocessing pipeline and a CNN-based classification model. The StanWiFi dataset was collected in

> S. Yousefi, H. Narui, S. Dayal, S. Ermon and S. Valaee, "A Survey on Behavior Recognition Using WiFi Channel State Information," in IEEE Communications Magazine, vol. 55, no. 10, pp. 98-104, Oct. 2017, doi: 10.1109/MCOM.2017.1700082.

The results obtained from the evaluation are stored in `02_RESULTS/01_STANWIFI/01_MODEL-REPORTS/reports.json` and can be loaded in the [Section 1.6](#summary) of this notebook.

## Imports

In [1]:
import os

import numpy as np

from tensorflow import keras
from tensorflow.keras import layers

from functions.filters import dbscan_filtering, wavelet_filtering
from functions.json import load_json, save_json
from functions.ml import clear_backend_and_seeds, cross_validation
from functions.report_metrics import metrics_summary

## Constants

In [2]:
RESULTS = '02_RESULTS'

STANWIFI_REPORTS = os.path.join(RESULTS, '01_STANWIFI', '01_MODEL-REPORTS', 'reports.json')
STANWIFI_LABELS = ['LIE DOWN', 'FALL', 'WALK', 'RUN', 'SITDOWN', 'STANDUP']

FOLDS = 10
BATCH_SIZE = 256
EPOCHS = 30

## Data loading

Follow the instructions provided in [`01_DATA/01_STANWIFI/README.md`](./01_DATA/01_STANWIFI/README.md) and then execute the following cell to load the data.

In [ ]:
from cross_vali_input_data import csv_import

x_bed, x_fall, x_pickup, x_run, x_sitdown, x_standup, x_walk, \
y_bed, y_fall, y_pickup, y_run, y_sitdown, y_standup, y_walk = csv_import()

## Data processing

- Apply DBSCAN filtering + DWT filtering
- Remove "pickup" activity from one-hot vectors

In [ ]:
x_bed_proc = np.apply_along_axis(lambda x: wavelet_filtering(dbscan_filtering(x)),1, x_bed)
x_fall_proc = np.apply_along_axis(lambda x: wavelet_filtering(dbscan_filtering(x)),1, x_fall)
x_run_proc = np.apply_along_axis(lambda x: wavelet_filtering(dbscan_filtering(x)),1, x_run)
x_sitdown_proc = np.apply_along_axis(lambda x: wavelet_filtering(dbscan_filtering(x)),1, x_sitdown)
x_standup_proc = np.apply_along_axis(lambda x: wavelet_filtering(dbscan_filtering(x)),1, x_standup)
x_walk_proc = np.apply_along_axis(lambda x: wavelet_filtering(dbscan_filtering(x)),1, x_walk)

In [ ]:
y_bed_proc = np.delete(y_bed, [0,4], axis=1)
y_fall_proc = np.delete(y_fall, [0,4], axis=1)
y_run_proc = np.delete(y_run, [0,4], axis=1)
y_sitdown_proc = np.delete(y_sitdown, [0,4], axis=1)
y_standup_proc = np.delete(y_standup, [0,4], axis=1)
y_walk_proc = np.delete(y_walk, [0,4], axis=1)

In [ ]:
x = np.vstack((x_bed_proc, x_fall_proc, x_run_proc, x_sitdown_proc, x_standup_proc, x_walk_proc))
x = np.transpose(x, axes=(0,2,1))
y = np.vstack((y_bed_proc, y_fall_proc, y_run_proc, y_sitdown_proc, y_standup_proc, y_walk_proc))

## 10-fold cross validation

> **WARNING**: Its execution can last several hours. You can instead load the results obtained by the authors in [Section 1.6](#summary)

In [2]:
def build_model():
    clear_backend_and_seeds()
    
    model = keras.Sequential([
        layers.Conv2D(filters=8, kernel_size=(3,10), input_shape=(90, 500, 1)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(),
        
        layers.Conv2D(filters=8, kernel_size=(3,10)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(),
        
        layers.Flatten(),
        
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(512, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(6, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
    return model

In [ ]:
reports = cross_validation(build_model, x, y, folds=FOLDS, batch_size=BATCH_SIZE, epochs=EPOCHS, labels=STANWIFI_LABELS)

In [ ]:
save_json(reports, STANWIFI_REPORTS)

<a id='summary'></a>
## Summary

In [3]:
reports = load_json(STANWIFI_REPORTS)

In [4]:
metrics_summary([reports], ['StanWiFi CV'])

,accuracy,precision,recall,f1-score
StanWiFi CV,0.967177,0.968634,0.967177,0.966725


The information presented in the above table corresponds to the one included in the **Table III** (StanWiFi > This work) of the paper.